In [3]:
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")
from braindecode.scripts.train_experiments import setup_logging
setup_logging()

Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, CuDNN 3007)


In [4]:
from braindecode.experiments.load import load_exp_and_model
from braindecode.veganlasagne.layer_util import print_layers, recompute_bnorm_layer_statistics
import lasagne

In [5]:
exp, model = load_exp_and_model('data/models/paper/ours/cnt/deep4/243', set_invalid_to_NaN=False)

2016-11-14 13:50:28,656 Setting n_sample preds automatically to 479
2016-11-14 13:50:28,660 Input window length is 522
2016-11-14 13:50:28,820 Setting n_sample preds automatically to 479
2016-11-14 13:50:28,822 Input window length is 522


"Conv2DAllColsLayer" is just a normal conv2d-layer which dynamically is assigned the filter size of 1x(4th input dim).
Should be replacable by normal Conv2DLayer if needed.

In [6]:
print_layers(model)

 0-InputLayer                                                         [None, 44, 1000, 1]
 1-DimshuffleLayer                                                    (None, 1, 1000, 44)
 2-DropoutLayer             0.000                                    
 3-Conv2DLayer              10x1                                      (None, 25, 991, 44)
 4-Conv2DAllColsLayer       1x44                                      (None, 25, 991, 1)
 5-BatchNormLayer                                     elu            
 6-Pool2DLayer              3x1                       max             (None, 25, 989, 1)
 7-StrideReshapeLayer           ::3 ::1                               (None, 25, 330, 1)
 8-NonlinearityLayer                                                 
 9-DropoutLayer             0.500                                    
10-Conv2DLayer              10x1                                      (None, 50, 321, 1)
11-BatchNormLayer                                     elu            
12-Pool2DLayer          

### Load and get data

In [7]:
exp.dataset.load()

2016-11-14 13:50:28,873 Load Training Set...
2016-11-14 13:50:52,690 Load Test Set...
2016-11-14 13:50:57,308 Clean Training Set...
2016-11-14 13:50:58,471 Rejected channels: []
2016-11-14 13:50:58,472 #Clean trials:     895
2016-11-14 13:50:58,472 #Rejected trials:  2
2016-11-14 13:50:58,472 Fraction Clean:    99.0%
2016-11-14 13:50:58,473 (from maxmin):     2
2016-11-14 13:50:58,473 (from var):        0
2016-11-14 13:50:58,783 Clean Test Set...
2016-11-14 13:50:58,988 Rejected channels: []
2016-11-14 13:50:58,989 #Clean trials:     160
2016-11-14 13:50:58,989 #Rejected trials:  0
2016-11-14 13:50:58,989 Fraction Clean:    100.0%
2016-11-14 13:50:58,989 (from maxmin):     0
2016-11-14 13:50:58,990 (from var):        0
2016-11-14 13:50:58,990 Create Cleaned Cnt Sets...
2016-11-14 13:51:01,037 Create sets from cleaned cnt...
2016-11-14 13:51:01,038 Preprocess continuous signal...
2016-11-14 13:51:26,547 Not doing any bandpass, since low 0 or None and high None
2016-11-14 13:51:51,606 Lo

In [8]:
datasets = exp.dataset_provider.get_train_merged_valid_test(exp.dataset)

In [9]:
test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))

### Just for showing how to make some predictions

In [10]:
from braindecode.veganlasagne.layers import create_pred_fn
pred_fn = create_pred_fn(model)

In [11]:
all_preds = []
all_targets = []
for inputs, targets in test_batches:
    all_preds.append(pred_fn(inputs))
    all_targets.append(targets)
all_targets = np.concatenate(all_targets)
all_preds = np.concatenate(all_preds)

In [12]:
print("Accuracy: {:.2f}%".format(100 * 
    np.mean(np.argmax(all_targets, axis=1) == np.argmax(all_preds, axis=1))))

Accuracy: 98.53%


In [2]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
#os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
